# 3 mosquito model: invasion probabilities, expected time until invasion, damping ratio, sojourn time

## Abby Barlow, University of Bath
## Ben Adams, University of Bath

Importing required libraries

In [1]:
import numpy as np
import itertools
from scipy.linalg import expm
import pylab as plt
import matplotlib
import sympy as sp

Importing required scripts

In [26]:
import importlib
import Rate_transitions
import Finding_sub_Q
import Lower_block_triangular
import Finding_full_Q
import Entropies
import Hughes_model
import Prob_absorb_to_each
import Time_absorb_wild_states

get_transition = Rate_transitions.get_transition_Hughes
getQk = Finding_sub_Q.getQk_Hughes
LBTQ = Lower_block_triangular.LBTQ_Hughes_comp
getQ = Finding_full_Q.getQ_Hughes
entropy = Entropies.entropy
F = Hughes_model.F_hughes
prob_reach_absorb = Prob_absorb_to_each.prob_reach_absorb_Hughes
absorb_time_wolb = Time_absorb_wild_states.absorb_time_wolb_Hughes
absorb_time_wild = Time_absorb_wild_states.absorb_time_wild_Hughes_comp

# scripts autosave, so no need to re-run code chunk after making changes
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Parameter values

In [6]:
# set some parameter values 

K = 3            # reproductive carrying capacity
d1 = 12/100      # wild-type death rate
d2 = 12/100      # Wolbachia death rate
xstar = 2        # imposed steady state (for wild type population)
k = 0.3          # Mosquito competition parameter
h = 0.19*100**k  # Mosquito competition paramete
b1 = round(d1/F(xstar,h,k,K),2) # wild-type per capita birth rate
phi = 85/100      # Wolbachia fitness
b2 = b1*phi      # Wolbachia  per capita fitness
v = 10/10        # probability of vertical transmission
u = 10/10        # probability of viable offspring


# create a dictionary to store all parameter values
params_dict = {'b1': b1,
              'b2': b2,
              'K': K,
              'h': h,
              'k': k,
              'd1': d1,
              'd2': d2,
              'u': u,
              'v': v,
              'phi': phi}

Construct a dictionary of all the state variables

In [7]:
# construct a dictionary that associated an integer index with each possible states, states are stored as an np.array - easier to apply mathematical operations than tuple 
max_pop = 3 # maximum household size
state_dict = {index: np.array((i, j)) for index, (i, j) in enumerate([(i, j) for i in range(max_pop + 1) for j in range(max_pop + 1) if i + j <= max_pop])}

Construct the full transition matrix

In [8]:
# construct a matrix Q for the transition rate q_ij betweeen states i and j
n_states = len(state_dict)  # total number of states

Q = getQ(state_dict,params_dict)
print(Q)

[[-0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.        ]
 [ 0.12       -0.2396846   0.1196846   0.          0.          0.
   0.          0.          0.          0.        ]
 [ 0.          0.24       -0.44097261  0.20097261  0.          0.
   0.          0.          0.          0.        ]
 [ 0.          0.          0.36       -0.36        0.          0.
   0.          0.          0.          0.        ]
 [ 0.12        0.          0.          0.         -0.26080541  0.
   0.          0.14080541  0.          0.        ]
 [ 0.          0.12        0.          0.          0.12       -0.3995959
   0.1004863   0.          0.05910959  0.        ]
 [ 0.          0.          0.12        0.          0.          0.24
  -0.36        0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.24        0.
   0.         -0.47643836  0.          0.23643836]
 [ 0.          0.          0.          0.          0.  

Constructing individual dictionaries of the communicating state classes and their respective sub-Q matrices

In [9]:
### S1 corresponds to the wild-type-only states, S2 to the Wolbachia-only and S3 the mixed states
state_dict_S1 = {index: np.array((i, 0)) for index, i in enumerate([i for i in range(1,max_pop + 1)])}
state_dict_S2 = {index: np.array((0, i)) for index, i in enumerate([i for i in range(1,max_pop + 1)])}
state_dict_S3 = {index: np.array((i,j)) for index, (i,j) in enumerate([(i, j) for i in range(1,max_pop + 1) for j in range(1,max_pop + 1) if i + j <= max_pop])}

# finding the sub-q matrices and their respective ordered lists of states in the class
# we will use these list to rearrange Q into lower block triangular form
Q1,key_list1 = getQk(state_dict_S1,state_dict,Q,params_dict)
Q2,key_list2 = getQk(state_dict_S2,state_dict,Q,params_dict)
Q3,key_list3 = getQk(state_dict_S3,state_dict,Q,params_dict)

Putting Q in lower block triangular form and constructing the reordered full state dictionary

In [10]:
Q_lower_block_triang, state_dict_relabel = LBTQ(Q,state_dict,state_dict_S1,state_dict_S2,state_dict_S3,max_pop,params_dict)
print('Re-ordered state dictionary is:', state_dict_relabel)
print()
print('Q in lower block triangular form is:', Q_lower_block_triang)

Re-ordered state dictionary is: {0: array([1, 0]), 1: array([2, 0]), 2: array([3, 0]), 3: array([0, 1]), 4: array([0, 2]), 5: array([0, 3]), 6: array([1, 2]), 7: array([1, 1]), 8: array([2, 1])}

Q in lower block triangular form is: [[-0.26080541  0.14080541  0.          0.          0.          0.
   0.          0.          0.        ]
 [ 0.24       -0.47643836  0.23643836  0.          0.          0.
   0.          0.          0.        ]
 [ 0.          0.36       -0.36        0.          0.          0.
   0.          0.          0.        ]
 [ 0.          0.          0.         -0.2396846   0.1196846   0.
   0.          0.          0.        ]
 [ 0.          0.          0.          0.24       -0.44097261  0.20097261
   0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.36       -0.36
   0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.12        0.
  -0.36        0.24        0.        ]
 [ 0.12        0.     

Find the QSD. This is the left eigenvector associated with the over all eigenvalue with minimal magitude normalised to sum to 1. This comes from S1.

In [11]:
# we take the transpose of Q so we obtain the left eigenvector not right
evals, evecs = np.linalg.eig(Q_lower_block_triang.T)  # all eigenvalues and eigenvectors
decay_indx = np.argmax([x for x in evals if x != 0])  # index for over all eigenvalue of minimal magnitude
uvec = evecs[:,decay_indx]                # the corresponding left eigenvector
quasi_stat_dist = uvec/np.sum(uvec)       # normalising to sum to 1
print('The QSD is', quasi_stat_dist)

The QSD is [0.3898769  0.34767302 0.26245008 0.         0.         0.
 0.         0.         0.        ]


Finding the sojourn time

In [12]:
soj_time = 0
for i in range(n_states-1):
    soj_time -= quasi_stat_dist[i]/Q_lower_block_triang[i,i]
print('expected time spent in a state is', soj_time)

expected time spent in a state is 2.9536572383304383


Finding the probabilities of reaching the Wolbachia-only state space (successful invasion) conditioning on each transient state

In [18]:
n_transient = len(state_dict_S3) 
prob_reach_wolb = np.zeros(n_transient)
for i in range(max_pop):
    absorb_state = np.array([0,i+1])
    prob_reach_wolb[:] += np.transpose(prob_reach_absorb(state_dict,state_dict_S3,absorb_state,params_dict)[0])[0]
print(prob_reach_wolb)
print(1-prob_reach_wolb)

[0.5235203  0.68234686 0.34901353]
[0.4764797  0.31765314 0.65098647]


Finding the expected times to invasion originating from each transient state

In [28]:
absorb_time_wolb(max_pop,np.ones(n_transient),params_dict)

array([5.02483039, 5.34792778, 7.80260816])

Damping ratio calculations

In [29]:
evals1, evecs1 = np.linalg.eig(Q1.T)
decay_param1 = np.max([x for x in evals1 if x != 0])
print('minimal magnitude eigenvalue of Q1', decay_param1)

evals2, evecs2 = np.linalg.eig(Q2.T)
decay_param2 = np.max([x for x in evals2 if x != 0])
print('minimal magnitude eigenvalue of Q2', decay_param2)

minimal magnitude eigenvalue of Q1 -0.04678522849353627
minimal magnitude eigenvalue of Q2 -0.05239675371166345


In [30]:
damp_ratio = np.exp(decay_param1-decay_param2) # damping ratio
print('the damping ratio is', damp_ratio)
print('The expected time until total probability mass of the wild-type-only states are')
print('10 times greater', np.log(10)/np.log(damp_ratio))
print('100 times greater', np.log(100)/np.log(damp_ratio))
print('1000 times greater', np.log(1000)/np.log(damp_ratio))

the damping ratio is 1.0056272993175468
The expected time until total probability mass of the wild-type-only states are
10 times greater 410.3314167698828
100 times greater 820.6628335397656
1000 times greater 1230.9942503096484
